In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [7]:
landmarks_frame = pd.read_csv('faces/face_landmarks.csv')

n = 61
img_name = landmarks_frame.iloc[n, 0]
landmarks = landmarks_frame.iloc[n, 1:].as_matrix()
landmarks = landmarks.astype('float').reshape(-1, 2)

print('Image name: {}'.format(img_name))
print('Landmarks shape: {}'.format(landmarks.shape))
print('First 4 Landmarks: {}'.format(landmarks[:4]))

Image name: 97308305_4b737d0873.jpg
Landmarks shape: (68, 2)
First 4 Landmarks: [[ 137.  157.]
 [ 139.  190.]
 [ 142.  223.]
 [ 149.  255.]]


In [6]:
def show_landmarks(image, landmarks):
    """Show image with landmarks"""
    plt.imshow(image)
    plt.scatter(landmarks[:, 0], landmarks[:, 1], s=10, marker='.', c='r')
    plt.pause(0.001)  # pause a bit so that plots are updated

plt.figure()
show_landmarks(io.imread(os.path.join('faces/', img_name)),
               landmarks)
plt.show()

In [8]:
img2 = io.imread('../test.JPG')

In [9]:
plt.imshow(img2)
plt.show()

In [28]:
class FaceLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:].as_matrix()
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [29]:
face_dataset = FaceLandmarksDataset(csv_file='faces/face_landmarks.csv',
                                    root_dir='faces/')

fig = plt.figure()

for i in range(len(face_dataset)):
    sample = face_dataset[i]

    print(i, sample['image'].shape, sample['landmarks'].shape)

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    show_landmarks(**sample)

    if i == 3:
        plt.show()
        break

0 (324, 215, 3) (68, 2)
1 (500, 333, 3) (68, 2)
2 (250, 258, 3) (68, 2)
3 (434, 290, 3) (68, 2)


In [38]:
class Dupa():
    def __init__(self, ds):
        self.ds = ds
    
    def __call__(self, sample):
        return len(sample)

In [39]:
a = [1,2,3,4,5,6]
b = [2,4]

In [40]:
d = Dupa(a)

In [41]:
e = d(b)

In [42]:
e

2

In [53]:
out = (5,4)
#assert isinstance(output, (int, tuple))
o = isinstance(out, (tuple))

In [54]:
o

True

In [55]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        # h and w are swapped for landmarks because for images,
        # x and y axes are axis 1 and 0 respectively
        landmarks = landmarks * [new_w / w, new_h / h]

        return {'image': img, 'landmarks': landmarks}


class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']

        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

        landmarks = landmarks - [left, top]

        return {'image': image, 'landmarks': landmarks}


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'landmarks': torch.from_numpy(landmarks)}

In [57]:
#scale = Rescale(256)
#crop = RandomCrop(128)
composed = transforms.Compose([Rescale(256),
                               RandomCrop(224)])

# Apply each of the above transforms on sample.
fig = plt.figure()
sample = face_dataset[65]
for i, tsfrm in enumerate([scale, crop, composed]):
    transformed_sample = tsfrm(sample)

    ax = plt.subplot(1, 3, i + 1)
    plt.tight_layout()
    ax.set_title(type(tsfrm).__name__)
    show_landmarks(**transformed_sample)

plt.show()

In [59]:
transformed_dataset = FaceLandmarksDataset(csv_file='faces/face_landmarks.csv',
                                           root_dir='faces/',
                                           transform=transforms.Compose([
                                               Rescale(256),
                                               RandomCrop(224),
                                               ToTensor()
                                           ]))

for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample['image'].size(), sample['landmarks'].size())

    if i == 3:
        break

0 torch.Size([3, 224, 224]) torch.Size([68, 2])
1 torch.Size([3, 224, 224]) torch.Size([68, 2])
2 torch.Size([3, 224, 224]) torch.Size([68, 2])
3 torch.Size([3, 224, 224]) torch.Size([68, 2])


In [60]:
dataloader = DataLoader(transformed_dataset, batch_size=4,
                        shuffle=True, num_workers=4)


# Helper function to show a batch
def show_landmarks_batch(sample_batched):
    """Show image with landmarks for a batch of samples."""
    images_batch, landmarks_batch = \
            sample_batched['image'], sample_batched['landmarks']
    batch_size = len(images_batch)
    im_size = images_batch.size(2)

    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))

    for i in range(batch_size):
        plt.scatter(landmarks_batch[i, :, 0].numpy() + i * im_size,
                    landmarks_batch[i, :, 1].numpy(),
                    s=10, marker='.', c='r')

        plt.title('Batch from dataloader')

for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['image'].size(),
          sample_batched['landmarks'].size())

    # observe 4th batch and stop.
    if i_batch == 3:
        plt.figure()
        show_landmarks_batch(sample_batched)
        plt.axis('off')
        plt.ioff()
        plt.show()
        break

0 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
1 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
2 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
3 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])


In [80]:
l = [1,2,3]
a = torch.tensor(l)
b = torch.tensor(l, requires_grad=True)
print(a.requires_grad, b.requires_grad)

False True


In [81]:
c = a+b


In [82]:
print(c.requires_grad)

True


In [83]:
with torch.no_grad():
    d = a+b

In [84]:
d

tensor([ 2,  4,  6])

In [85]:
c

tensor([ 2,  4,  6])

In [86]:
print(d.requires_grad)

False


In [87]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [92]:
lin = nn.Linear(5, 3)  # maps from R^5 to R^3, parameters A, b
# data is 2x5.  A maps from 5 to 3... can we map "data" under A?
data = torch.randn(2, 5)
print(data)
print(lin(data))  # yes

tensor([[ 0.5308, -0.4645,  0.7340, -0.4394,  0.9255],
        [ 0.8003, -0.4376, -0.4595,  1.5414,  1.5618]])
tensor([[-0.4434,  0.5226, -0.5102],
        [-0.0846, -0.4863,  0.1600]])


In [98]:
data = torch.ones(5)
print(data)
print(data.size())
print(F.softmax(data))
print(F.softmax(data, dim=0))
print(F.softmax(data, dim=0).sum())  # Sums to 1 because it is a distribution!
print(F.log_softmax(data, dim=0))  # theres also log_softmax

tensor([ 1.,  1.,  1.,  1.,  1.])
torch.Size([5])
tensor([ 0.2000,  0.2000,  0.2000,  0.2000,  0.2000])
tensor([ 0.2000,  0.2000,  0.2000,  0.2000,  0.2000])
tensor(1.)
tensor([-1.6094, -1.6094, -1.6094, -1.6094, -1.6094])


In [102]:
print(lin)

Linear(in_features=5, out_features=3, bias=True)


In [110]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

In [148]:
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2


class BoWClassifier(nn.Module):  # inheriting from nn.Module!

    def __init__(self, num_labels, vocab_size):
        # calls the init function of nn.Module.  Dont get confused by syntax,
        # just always do it in an nn.Module
        super(BoWClassifier, self).__init__()

        # Define the parameters that you will need.  In this case, we need A and b,
        # the parameters of the affine mapping.
        # Torch defines nn.Linear(), which provides the affine map.
        # Make sure you understand why the input dimension is vocab_size
        # and the output is num_labels!
        self.linear = nn.Linear(vocab_size, num_labels)

        # NOTE! The non-linearity log softmax does not have parameters! So we don't need
        # to worry about that here

    def forward(self, bow_vec):
        # Pass the input through the linear layer,
        # then pass that through log_softmax.
        # Many non-linearities and other functions are in torch.nn.functional
        return F.log_softmax(self.linear(bow_vec), dim=1)


def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)


def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])


model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

# the model knows its parameters.  The first output below is A, the second is b.
# Whenever you assign a component to a class variable in the __init__ function
# of a module, which was done with the line
# self.linear = nn.Linear(...)
# Then through some Python magic from the PyTorch devs, your module
# (in this case, BoWClassifier) will store knowledge of the nn.Linear's parameters
for param in model.parameters():
    print(param)

# To run the model, pass in a BoW vector
# Here we don't need to train, so the code is wrapped in torch.no_grad()
print("-----")
with torch.no_grad():
    sample = data[0]
    #print(sample)
    #print(sample[0])
    bow_vector = make_bow_vector(sample[0], word_to_ix)
    #print(bow_vector)
    log_probs = model(bow_vector)
    print(log_probs)
    
    
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}    

{'not': 17, 'una': 13, 'lost': 21, 'get': 20, 'gusta': 1, 'la': 4, 'Give': 6, 'good': 19, 'on': 25, 'it': 7, 'cafeteria': 5, 'a': 18, 'No': 9, 'is': 16, 'Yo': 23, 'to': 8, 'que': 11, 'me': 0, 'comer': 2, 'buena': 14, 'creo': 10, 'at': 22, 'si': 24, 'idea': 15, 'sea': 12, 'en': 3}
Parameter containing:
tensor([[ 0.0611,  0.0291, -0.0714,  0.1438,  0.0290,  0.1264,  0.1894,
         -0.0439,  0.1381, -0.0398,  0.1770,  0.1544, -0.0703,  0.0070,
         -0.0856, -0.0793, -0.0397, -0.0013, -0.0684, -0.1851, -0.1186,
         -0.0168, -0.0626,  0.1457,  0.1594,  0.1249],
        [ 0.0868, -0.1555,  0.1853, -0.1728, -0.0645, -0.1380,  0.0705,
         -0.1632, -0.0967, -0.1246, -0.1205, -0.0916,  0.1928, -0.0160,
          0.0598, -0.1947,  0.1744, -0.0427, -0.1800,  0.1352, -0.0530,
          0.1781, -0.1332, -0.1821, -0.1473,  0.0850]])
Parameter containing:
tensor([-0.0220, -0.1817])
-----
tensor([[-0.3913, -1.1276]])


In [153]:
# Run on test data before we train, just to see a before-and-after
with torch.no_grad():
    for instance, label in test_data:
        bow_vec = make_bow_vector(instance, word_to_ix)
        log_probs = model(bow_vec)
        print(log_probs)

# Print the matrix column corresponding to "creo"
print("creo prediction")
print(next(model.parameters())[:, word_to_ix["creo"]])

tensor([[-0.2331, -1.5707]])
tensor([[-0.7526, -0.6370]])
creo prediction
tensor([ 0.1770, -0.1205])


In [169]:
next(model.parameters())[:, word_to_ix["it"]]

tensor([-0.0439, -0.1632])

In [171]:
model.parameters()[:, word_to_ix["it"]]

TypeError: 'generator' object is not subscriptable

In [176]:
a = [1,2,3]
b = iter(a)
next(b)

1

In [177]:
next(model.parameters())

Parameter containing:
tensor([[ 0.0611,  0.0291, -0.0714,  0.1438,  0.0290,  0.1264,  0.1894,
         -0.0439,  0.1381, -0.0398,  0.1770,  0.1544, -0.0703,  0.0070,
         -0.0856, -0.0793, -0.0397, -0.0013, -0.0684, -0.1851, -0.1186,
         -0.0168, -0.0626,  0.1457,  0.1594,  0.1249],
        [ 0.0868, -0.1555,  0.1853, -0.1728, -0.0645, -0.1380,  0.0705,
         -0.1632, -0.0967, -0.1246, -0.1205, -0.0916,  0.1928, -0.0160,
          0.0598, -0.1947,  0.1744, -0.0427, -0.1800,  0.1352, -0.0530,
          0.1781, -0.1332, -0.1821, -0.1473,  0.0850]])

In [178]:
next(model.parameters())

Parameter containing:
tensor([[ 0.0611,  0.0291, -0.0714,  0.1438,  0.0290,  0.1264,  0.1894,
         -0.0439,  0.1381, -0.0398,  0.1770,  0.1544, -0.0703,  0.0070,
         -0.0856, -0.0793, -0.0397, -0.0013, -0.0684, -0.1851, -0.1186,
         -0.0168, -0.0626,  0.1457,  0.1594,  0.1249],
        [ 0.0868, -0.1555,  0.1853, -0.1728, -0.0645, -0.1380,  0.0705,
         -0.1632, -0.0967, -0.1246, -0.1205, -0.0916,  0.1928, -0.0160,
          0.0598, -0.1947,  0.1744, -0.0427, -0.1800,  0.1352, -0.0530,
          0.1781, -0.1332, -0.1821, -0.1473,  0.0850]])